In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from lightgbm import LGBMClassifier

from imblearn.over_sampling import RandomOverSampler

from scipy.stats import loguniform, uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from feature_engineering import *

import warnings
warnings.filterwarnings('ignore')

c:\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.6) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Import data

In [3]:
X_train = pd.read_excel("data/model_inputs/X_train.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test = pd.read_excel("data/model_inputs/X_test.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train = pd.read_excel("data/model_inputs/y_train.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test = pd.read_excel("data/model_inputs/y_test.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1450, 11), (168, 11), (1450, 1), (168, 1))

Tuning for best combination

In [4]:
def random_oversampler(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='not majority')
    X_over, y_over = oversample.fit_resample(X_train, y_train)
    return X_over, y_over
    
lgbm_distribution = {
    'learning_rate': loguniform(0.01, 0.99),
    'max_depth': randint(1, 10),
    'min_child_weight': randint(1, 10),
    'boosting':  ['gbdt', 'dart'],
    'min_gain_to_split': uniform(0, 10),
    'num_leaves': randint(10, 300, 10),
    'subsample': uniform(0.5, 0.4),
    'colsample_bytree': uniform(0.5, 0.4),
    'colsample_bynode': uniform(0.5, 0.4),
    'reg_lambda': uniform(0, 1),
    'reg_alpha': randint(0, 100)
}

In [5]:
param, accuracy, precision, recall, f1 = [], [], [], [], []

# best oversampling: random_oversampling
X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)

# tuning
lgbm = LGBMClassifier()
lgbm_clf = RandomizedSearchCV(lgbm, lgbm_distribution, n_iter=200, scoring=['accuracy', 'recall_weighted', 'precision_weighted', 'f1_weighted'], refit='f1_weighted', random_state=42)
lgbm_search = lgbm_clf.fit(X_train_oversampled, y_train_oversampled)

# update columns
param.append(lgbm_search.cv_results_['params'])
accuracy.append(lgbm_search.cv_results_['mean_test_accuracy'])
precision.append(lgbm_search.cv_results_['mean_test_precision_weighted'])
recall.append(lgbm_search.cv_results_['mean_test_recall_weighted'])
f1.append(lgbm_search.cv_results_['mean_test_f1_weighted'])

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=4.458327528535912, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.458327528535912
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=4.458327528535912, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.458327528535912
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=4.458327528535912, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.458327528535912
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=4.458327528535912, min_split_gain=0.0 will be ignored. Current val

In [6]:
results = pd.DataFrame({
    'model': "LGBM",
    'parameters': np.array(param).flatten(),
    'accuracy': np.array(accuracy).flatten(),
    'precision': np.array(precision).flatten(),
    'recall': np.array(recall).flatten(),
    'f1': np.array(f1).flatten(),
}).sort_values(by="f1", ascending=False).reset_index(drop=True)

In [7]:
results.head(10)

,model,parameters,accuracy,precision,recall,f1
0,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.571...",0.942875,0.952178,0.942875,0.939993
1,LGBM,"{'boosting': 'gbdt', 'colsample_bynode': 0.618...",0.940470,0.951183,0.940470,0.936416
2,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.551...",0.938297,0.948973,0.938297,0.934392
3,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.516...",0.929859,0.940341,0.929859,0.926337
4,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.798...",0.930586,0.944079,0.930586,0.924803
5,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.872...",0.926238,0.941667,0.926238,0.921157
6,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.553...",0.920946,0.932744,0.920946,0.915853
7,LGBM,"{'boosting': 'gbdt', 'colsample_bynode': 0.659...",0.917561,0.930825,0.917561,0.912460
8,LGBM,"{'boosting': 'dart', 'colsample_bynode': 0.807...",0.916599,0.931241,0.916599,0.910525
9,LGBM,"{'boosting': 'gbdt', 'colsample_bynode': 0.846...",0.917562,0.934328,0.917562,0.910155


Best Model

In [8]:
best_params = results.iloc[0]['parameters']
best_model = LGBMClassifier(**best_params)
best_model.fit(X_train_oversampled, y_train_oversampled)
best_model

LGBMClassifier(boosting='dart', colsample_bynode=0.5713490337229319,
               colsample_bytree=0.5116285521375815,
               learning_rate=0.02152989018360296, max_depth=9,
               min_child_weight=4, min_gain_to_split=4.023271246865559,
               num_leaves=90, reg_alpha=3, reg_lambda=0.5101097124988252,
               subsample=0.7893619062449145)

In [9]:
def print_results(actual, predictions):
    print(f"accuracy: {accuracy_score(actual, predictions)}")
    print(f"precision: {precision_score(actual, predictions, average='weighted')}")
    print(f"recall: {recall_score(actual, predictions, average='weighted')}")
    print(f"f1: {f1_score(actual, predictions, average='weighted')}")
    print(f"confusion matrix:\n{confusion_matrix(actual, predictions)}")

Predict on test set

In [10]:
test_pred = best_model.predict(X_test)
print_results(y_test, test_pred)

accuracy: 0.8214285714285714
precision: 0.7939834616481324
recall: 0.8214285714285714
f1: 0.7463652027796764
confusion matrix:
[[  1  20   0]
 [  0 137   0]
 [  0  10   0]]


Predict on full data

In [11]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

full_pred = best_model.predict(X_full)
print_results(y_full, full_pred)

accuracy: 0.9425216316440049
precision: 0.9525822799823906
recall: 0.9425216316440049
f1: 0.946525953675668
confusion matrix:
[[  40   20    0]
 [  29 1457   34]
 [   0   10   28]]


Business metric

In [12]:
whole_df_x = pd.concat([X_train, X_test])
whole_df_x['year'] = whole_df_x.index.year

In [76]:
business_metric_results = pd.DataFrame(columns=['exp_strategy_annual_return', 'exp_benchmark_annual_return'])

for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_data = whole_df_x[whole_df_x['year'] == year]
    year_data = year_data.drop(['year'], axis = 1)
    predict_x = best_model.predict(np.array(year_data))
    predictions = predict_x
    df_pred = pd.DataFrame({'prediction':predictions}, index=year_data.index)
    dates = df_pred.index

    if year == 2022:
        end_date = "2022-09-02"
    else:
        end_date = str(year+1) + "-01-01"
    df_prices = yf.download("^GSPC", start=dates[0], end=end_date)[['Adj Close']]
    df_prices = df_prices.reindex(df_pred.index)

    # create positions column
    positions = []
    prev = 0
    for i in range(len(df_pred)):
        if df_pred.iloc[i]['prediction'] == 0:
            positions.append(prev)
        else:
            prev = df_pred.iloc[i]['prediction']
            positions.append(prev)

    df_business = pd.DataFrame()
    df_business['stock_daily_log_return'] = np.log(df_prices /df_prices.shift(1))['Adj Close']
    df_business['prediction'] = df_pred['prediction']
    df_business['position'] = positions
    df_business['benchmark'] = 1 # long and hold strategy
    df_business["strategy_Returns"] = df_business["stock_daily_log_return"] * df_business["position"].shift(1)
    df_business["benchmark_Returns"] = df_business["stock_daily_log_return"] * df_business["benchmark"].shift(1)

    # Annual Mean Returns or Expected returns
    expected_strategy_annual_return = np.exp(df_business['strategy_Returns'].mean() * 252) - 1 
    expected_benchmark_annual_return = np.exp(df_business['benchmark_Returns'].mean() * 252) - 1 
    business_metric_results.loc[year] = [expected_strategy_annual_return, expected_benchmark_annual_return]

business_metric_results

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,exp_strategy_annual_return,exp_benchmark_annual_return
2016,0.000000,0.107616
2017,0.000000,0.185753
2018,-0.123474,-0.070634
2019,0.091016,0.288443
2020,0.648653,0.152929
2021,0.219549,0.289230
2022,0.151850,-0.249185
